# Build a model with Gensim word embeddings
This will be mostly the same as the other model but instead of having it's own embedding layer it will contain embeddings from Gensim's `Word2Vec`

In [2]:
DATASETS_DIR = "../datasets"
MODELS_DIR = "../models"

In [3]:
# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')

  # read all the text
  text = file.read()

  # close the file
  file.close()
  return text

in_filename = f"{DATASETS_DIR}/content_sequences.txt"
doc = load_doc(in_filename)
lines = doc.split('\n')
lines[:4]

['mufundisi wekumakokoba kubulawayo akashamisa vagari venzvimbo iyi apo akarova muroja wake uyo anove asina kupfeka sechirango chekuridza mimhanzi zvine ruzha rwakanyanyisa apo iye aive pakati pekuitisa svondo mufundisi uye muvambi wechechi yetsime rase birthsider pastor khumbulani mzizi uyo aiitisa chechi pamba pake akarova sikhululiwe dube apo airidza mimhanzi zvine ruzha pawairesi',
 'wekumakokoba kubulawayo akashamisa vagari venzvimbo iyi apo akarova muroja wake uyo anove asina kupfeka sechirango chekuridza mimhanzi zvine ruzha rwakanyanyisa apo iye aive pakati pekuitisa svondo mufundisi uye muvambi wechechi yetsime rase birthsider pastor khumbulani mzizi uyo aiitisa chechi pamba pake akarova sikhululiwe dube apo airidza mimhanzi zvine ruzha pawairesi yake',
 'kubulawayo akashamisa vagari venzvimbo iyi apo akarova muroja wake uyo anove asina kupfeka sechirango chekuridza mimhanzi zvine ruzha rwakanyanyisa apo iye aive pakati pekuitisa svondo mufundisi uye muvambi wechechi yetsime r

## Encode sequences with using `Word2Vec` word index

In [7]:
# define a function to encode text sequences using Word2Vec word index
# instead of Tokenizer's word index
# This is because one of the models will use external word embeddings
def encode_text_with_word2vec(input_text, model, oov_token="<OOV>"):
  # OOV configuration
  oov_index = -1
  oov_token = "<OOV>"

  # list of encoded sequences
  sequences = list()

  for text in input_text:
    # split the text into words (Tokenize it)
    words = text.split()
    sequence = list()

    # map words to indeces using Word2Vec's word index
    for word in words:
      if word in model.wv.key_to_index:
        sequence.append(model.wv.key_to_index[word])
      else:
        # Handle OOV words by mapping them to a special token (oov_token in this case)
        sequence.append(model.wv.key_to_index.get(oov_token,0)) # 0 for unknown words

    sequences.append(sequence)

  return sequences

In [9]:
# load Word2Vec model
from gensim.models import Word2Vec

w2v_model = Word2Vec.load(f"{MODELS_DIR}/word2vec.model")

In [10]:
sequences = encode_text_with_word2vec(lines, w2v_model)

## Sequence inputs and output
Now that there are encoded input sequences, they need to be seperated into input($X$) and output($y$). This can be done via array slicing.

In [12]:
from numpy import array
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [13]:
X.shape

(30430, 50)

In [14]:
y.shape

(30430,)

After seperating, each output word needs to be one hot encoded. This means converting it from an integer to a vector of 0 values, one for each word in the vocabulary, with a 1 to indicate the specific word at the of the words integer value. This is so that the model learns to predict the probability distribution for the next word and the ground truth from which to learn from is 0 for all words except the actual word that comes next.

In [16]:
# one-hot encode the output word
from keras.utils import to_categorical

vocab_size = len(w2v_model.wv.key_to_index)

y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
seq_length

50

## Fit the model

In [48]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional, Dropout
from keras.layers import Embedding
from keras.utils import plot_model

# define the model
def define_model_with_embedding(vocab_size, seq_length, summary=True):
  model = Sequential()
  model.add(Embedding(len(w2v_model.wv.key_to_index), 100, weights=[w2v_model.wv.vectors], trainable=False))
  model.add(Bidirectional(LSTM(100)))
  model.add(Dropout(0.2))
  model.add(Dense(100, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(vocab_size, activation='softmax'))

  # compile network
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # summarize the model
  if summary:
    model.summary()
    model_img_path = f'{MODELS_DIR}/model_with_gensim_embedding.png'
    plot_model(model, to_file=model_img_path, show_shapes=True)
  return model

In [37]:
# define model
model_with_gensim_embedding = define_model_with_embedding(vocab_size, seq_length)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 100)         1089700   
                                                                 
 bidirectional_9 (Bidirecti  (None, 200)               160800    
 onal)                                                           
                                                                 
 dropout_9 (Dropout)         (None, 200)               0         
                                                                 
 dense_12 (Dense)            (None, 100)               20100     
                                                                 
 dropout_10 (Dropout)        (None, 100)               0         
                                                                 
 dense_13 (Dense)            (None, 10897)             1100597   
                                                      

In [38]:
# fit model
model_with_gensim_embedding.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
238/238 [==============================] - 47s 175ms/step - loss: 8.6316 - accuracy: 0.0219
Epoch 2/100
238/238 [==============================] - 42s 174ms/step - loss: 7.9931 - accuracy: 0.0300
Epoch 3/100
238/238 [==============================] - 40s 169ms/step - loss: 7.5568 - accuracy: 0.0411
Epoch 4/100
238/238 [==============================] - 40s 170ms/step - loss: 7.1347 - accuracy: 0.0504
Epoch 5/100
238/238 [==============================] - 41s 170ms/step - loss: 6.6623 - accuracy: 0.0592
Epoch 6/100
238/238 [==============================] - 41s 171ms/step - loss: 6.1146 - accuracy: 0.0694
Epoch 7/100
238/238 [==============================] - 41s 171ms/step - loss: 5.4800 - accuracy: 0.0840
Epoch 8/100
238/238 [==============================] - 41s 173ms/step - loss: 4.8292 - accuracy: 0.1081
Epoch 9/100
238/238 [==============================] - 41s 174ms/step - loss: 4.2469 - accuracy: 0.1563
Epoch 10/100
238/238 [==============================] - 41s 172m

In [40]:
from pickle import dump

# save the model to file
model_with_gensim_embedding.save(f'{MODELS_DIR}/model_with_gensim_embedding.keras')

## Training with cross validation

In [46]:
from sklearn.model_selection import KFold
import numpy as np

In [47]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
# define 5-fold cross validation test harness
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

cvscores = []
for train, test in kfold.split(X, y):
    # compile the model
    model_with_embedding = define_model_with_embedding(vocab_size, seq_length, summary=False)
    
    # Fit the model
    model_with_embedding.fit(X[train], y[train], epochs=20, batch_size=128)
    
    # Evaluate the model
    model_with_embedding.evaluate(X[test], y[test])
    print("Accuracy = %s: %.2f%%" % (model_with_embedding.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("Mean Accuracy = %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))